In [30]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

file_paths = glob.glob('unclean_data/*.csv')

In [35]:
indicator_names = [
    "Adult Literacy Rate",
    "ICT Service Exports",
    "R&D Expenditure",
    "Journal Articles",
    "Patent Applications",
    "Mobile Subscriptions",
    "Fixed Broadband Subscriptions",
    "Secure Internet Servers (per million)",
    "Secure Internet Servers (alt)",
    "Internet Users",
    "ICT Goods Imports",
    "High-Tech Exports",
    "Tertiary Education Enrollment"
]

final_data = {}

for file_path, indicator in zip(file_paths, indicator_names):
    df = pd.read_csv(file_path, skiprows=4)
    
    df = df.drop(columns=['Indicator Name', 'Indicator Code'], errors='ignore')
    
    for _, row in df.iterrows():
        country = row['Country Name'].strip()
        if country not in final_data:
            final_data[country] = {}

        year_data = row.drop(labels=['Country Name', 'Country Code'], errors='ignore')
        year_data = year_data.dropna()

        if not year_data.empty:
            latest_value = year_data.iloc[-1]
            final_data[country][indicator] = latest_value
        else:
            final_data[country][indicator] = "NONE"

reshaped_df = pd.DataFrame.from_dict(final_data, orient='index')
reshaped_df.index.name = "Country"

reshaped_df.to_csv('cleaning_process/reshaped_data.csv', index=True)

print(reshaped_df.head())

                            Adult Literacy Rate ICT Service Exports  \
Country                                                               
Aruba                                  0.661533                NONE   
Africa Eastern and Southern            6.106573            0.586779   
Afghanistan                            7.828516                NONE   
Africa Western and Central             5.691769            0.151699   
Angola                                 45.41476             0.03229   

                            R&D Expenditure Journal Articles  \
Country                                                        
Aruba                                  NONE              1.0   
Africa Eastern and Southern        32147.61             NONE   
Afghanistan                          169.19             NONE   
Africa Western and Central         16427.93             NONE   
Angola                                44.99              2.0   

                            Patent Applications Mobil